# Part II: Visualizations latent space with toolkit 'Kerac'

CAS AML Module3 by Beatriz Vidondo

Using keract: https://github.com/philipperemy/keract
Keract toolkit allows to “get the activations (outputs) and gradients for each layer of Keras models” (Rémy, 2019)

Code adapted from 
https://www.machinecurve.com/index.php/2019/12/26/how-to-visualize-the-encoded-state-of-an-autoencoder-with-keras/#visualizing-the-encoded-state-what-we-want-to-achieve

In [1]:
import keras
from keras.layers import Input, Dense
from keras.datasets import mnist
from keras.models import Model
from keract import get_activations, display_activations
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

This time I use a larger number of neurons in the latent space: 50

In [2]:
# Model configuration
img_width, img_height = 28, 28
initial_dimension = img_width * img_height
batch_size = 24
no_epochs = 10
validation_split = 0.1
verbosity = 1
encoded_dim = 50 #number of nodes in the latent space

In [3]:
data_dir_str = 'C:/Users/beatriz/Documents/Python/MyModule3/data/squarecircletriang/shapes/'

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir_str,
    #label_mode = None,
    validation_split=validation_split,
    subset="training",
    color_mode='grayscale',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir_str,
    #label_mode = None,
    validation_split=validation_split,
    subset="validation",
    color_mode='grayscale',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 300 files belonging to 3 classes.
Using 270 files for training.
Found 300 files belonging to 3 classes.
Using 30 files for validation.


In [4]:
# store and rescale the image data as well
train_images4D=np.concatenate(list(map(lambda x: x[0].numpy()/255, train_ds)))
test_images4D=np.concatenate(list(map(lambda x: x[0].numpy()/255, test_ds)))
print(train_images4D.shape)
print(test_images4D.shape)

(270, 28, 28, 1)
(30, 28, 28, 1)


In [5]:
# in case of color pics, select only the blue/last channel
#train_images = train_images4D[:,:,:,2]
#in case of grayscale pics, take the one color channel
train_images = train_images4D[:,:,:,0]
print(train_images.shape)
#view the data of first picture
#train3D[0]
test_images = test_images4D[:,:,:,0]
print(test_images.shape)

(270, 28, 28)
(30, 28, 28)


In [6]:
# Reshape data
input_train = train_images.reshape(train_images.shape[0], initial_dimension)
input_test = test_images.reshape(test_images.shape[0], initial_dimension)
input_shape = (initial_dimension, )
print(input_train.shape); print(input_test.shape)

(270, 784)
(30, 784)


## Version 1: define model with Keras Functional API

In [7]:
# Define the layers
inputs = Input(shape=input_shape)
encoding_layer = Dense(encoded_dim, activation='relu', kernel_initializer='he_normal')(inputs)
decoding_layer = Dense(initial_dimension, activation='sigmoid')(encoding_layer)

In [8]:
# Instantiate the autoencoder
autoencoder = Model(inputs, decoding_layer, name='full_autoencoder')

In [9]:
# Instantiate the encoder
encoder = Model(inputs, encoding_layer, name='encoder')

In [10]:
# Instantiate the decoder
encoded_input = Input(shape=(encoded_dim, ))
final_ae_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, final_ae_layer(encoded_input), name='decoder')

In [11]:
# Compile the autoencoder
encoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [12]:
# Give us some insights
autoencoder.summary()
encoder.summary()
decoder.summary()

Model: "full_autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 784)               39984     
Total params: 79,234
Trainable params: 79,234
Non-trainable params: 0
_________________________________________________________________
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 50)                39250     
Total params: 39,250
Trainabl

In [13]:
# Fit data
autoencoder.fit(input_train, input_train, epochs=no_epochs, batch_size=batch_size, validation_split=validation_split)

Epoch 1/10

11/11 [==============================] - 1s 20ms/step - loss: 0.5850 - val_loss: 0.3844
Epoch 2/10

11/11 [==============================] - 0s 7ms/step - loss: 0.2992 - val_loss: 0.2230
Epoch 3/10

11/11 [==============================] - 0s 7ms/step - loss: 0.2117 - val_loss: 0.1925
Epoch 4/10

11/11 [==============================] - 0s 6ms/step - loss: 0.1985 - val_loss: 0.1900
Epoch 5/10

11/11 [==============================] - 0s 7ms/step - loss: 0.1960 - val_loss: 0.1877
Epoch 6/10

11/11 [==============================] - 0s 7ms/step - loss: 0.1948 - val_loss: 0.1871
Epoch 7/10

11/11 [==============================] - 0s 5ms/step - loss: 0.1943 - val_loss: 0.1869
Epoch 8/10

11/11 [==============================] - 0s 7ms/step - loss: 0.1940 - val_loss: 0.1867
Epoch 9/10

11/11 [==============================] - 0s 6ms/step - loss: 0.1941 - val_loss: 0.1861
Epoch 10/10

11/11 [==============================] - 0s 5ms/step - loss: 0.1941 - val_loss: 0.1860


In [21]:
# =============================================
# Take a sample for visualization purposes
# =============================================
input_sample = input_test[:1]
reconstruction = autoencoder.predict([input_sample])

In [22]:
# =============================================
# Visualize input-->reconstruction
# =============================================
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(6, 3.5)
input_sample_reshaped = input_sample.reshape((img_width, img_height))
reconsstruction_reshaped = reconstruction.reshape((img_width, img_height))
axes[0].imshow(input_sample_reshaped) 
axes[0].set_title('Original image')
axes[1].imshow(reconsstruction_reshaped)
axes[1].set_title('Reconstruction')
plt.show()

In [23]:
# =============================================
# Visualize encoded state with Keract
# =============================================
activations = get_activations(encoder, input_sample)
display_activations(activations, cmap="gray", save=False)

input_1 (1, 784) 
dense (1, 50) 


## Version 2: define model with Keras Sequential API

In [17]:
import keras
from keras.layers import Dense
from keras.datasets import mnist
from keras.models import Sequential
from keract import get_activations, display_activations
import matplotlib.pyplot as plt
from keras import backend as K

In [18]:
# Model configuration
img_width, img_height = 28, 28
initial_dimension = img_width * img_height
batch_size = 24
no_epochs = 100
validation_split = 0.1
verbosity = 1
encoded_dim = 50

In [19]:
# Define the 'autoencoder' full model
autoencoder = Sequential()
autoencoder.add(Dense(encoded_dim, activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
autoencoder.add(Dense(initial_dimension, activation='sigmoid'))

In [20]:
# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Give us some insights
autoencoder.summary()

# Fit data
autoencoder.fit(input_train, input_train, epochs=no_epochs, batch_size=batch_size, validation_split=validation_split)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 50)                39250     
_________________________________________________________________
dense_3 (Dense)              (None, 784)               39984     
Total params: 79,234
Trainable params: 79,234
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100

11/11 [==============================] - 0s 15ms/step - loss: 0.6363 - val_loss: 0.4912
Epoch 2/100

11/11 [==============================] - 0s 6ms/step - loss: 0.3852 - val_loss: 0.2681
Epoch 3/100

11/11 [==============================] - 0s 6ms/step - loss: 0.2398 - val_loss: 0.2042
Epoch 4/100

11/11 [==============================] - 0s 6ms/step - loss: 0.2034 - val_loss: 0.1916
Epoch 5/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1971 - val_loss: 0.1884
Epoch 6/100



Epoch 47/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1924 - val_loss: 0.1851
Epoch 48/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1922 - val_loss: 0.1849
Epoch 49/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1922 - val_loss: 0.1848
Epoch 50/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1923 - val_loss: 0.1853
Epoch 51/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1919 - val_loss: 0.1856
Epoch 52/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1917 - val_loss: 0.1846
Epoch 53/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1917 - val_loss: 0.1845
Epoch 54/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1916 - val_loss: 0.1843
Epoch 55/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1916 - val_loss: 0.1842
Epoch 56/100

11/11 [==============================] - 0s 5ms/step - loss: 0.1914 - val_los

11/11 [==============================] - 0s 7ms/step - loss: 0.1802 - val_loss: 0.1749
Epoch 97/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1800 - val_loss: 0.1751
Epoch 98/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1794 - val_loss: 0.1737
Epoch 99/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1791 - val_loss: 0.1748
Epoch 100/100

11/11 [==============================] - 0s 6ms/step - loss: 0.1787 - val_loss: 0.1729


In [24]:
# =============================================
# Take a sample for visualization purposes
# =============================================
input_sample = input_test[:1]
reconstruction = autoencoder.predict([input_sample])

# =============================================
# Visualize input-->reconstruction
# =============================================
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(6, 3.5)
input_sample_reshaped = input_sample.reshape((img_width, img_height))
reconsstruction_reshaped = reconstruction.reshape((img_width, img_height))
axes[0].imshow(input_sample_reshaped) 
axes[0].set_title('Original image')
axes[1].imshow(reconsstruction_reshaped)
axes[1].set_title('Reconstruction')
plt.show()

In [25]:
# =============================================
# Visualize encoded state with Keract
# =============================================
activations = get_activations(autoencoder, input_sample)
display_activations(activations, cmap="gray", save=False)

dense_2_input (1, 784) 
dense_2 (1, 50) 
dense_3 (1, 784) 


We get an extra visualization: the output by the decoder before it’s reshaped into 28 x 28 pixels format,
but this is due to the relative inflexibility of the Sequential API